In [34]:
import cv2
import os 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [37]:
def extract_sift_features(image):
    # Get keypoints and discriptors
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(image, None)
    return descriptors

In [72]:
X_train_1 = []
X_test_1 = []
y_train_1 = []
y_test_1 = []

count = 0
for filename in os.listdir('images/database'):
    if filename == '.DS_Store':
        continue
    img = plt.imread('images/database/' + filename)
    # img = cv2.resize(img, (1500, 1500))
    X_train_1.append(img)
    y_train_1.append(count//8)
    count += 1

count = 0
for filename in os.listdir('images/test'):
    if filename == '.DS_Store':
        continue
    img = plt.imread('images/test/' + filename)
    # img = cv2.resize(img, (1500, 1500))
    X_test_1.append(img)
    y_test_1.append(count//2)
    count += 1

In [73]:
sift_features_train = [extract_sift_features(img) for img in X_train_1]
sift_features_test = [extract_sift_features(img) for img in X_test_1]

In [74]:
def BF_FeatureMatcher(des1,des2):    
    # Find the Euclidean distance between the 2 discriptors
    brute_force = cv2.BFMatcher(cv2.NORM_L2,crossCheck=True)
    no_of_matches = brute_force.match(des1,des2)
    
    no_of_matches = sorted(no_of_matches,key=lambda x:x.distance)
    return no_of_matches

In [75]:
total_tests = len(X_test_1)
total_train = len(X_train_1)
preds = []

for i in range(total_tests):
    found_label, min_distance = 0, float("inf")
    for j in range(total_train):
        matches = BF_FeatureMatcher(sift_features_test[i], sift_features_train[j])
        if min_distance > matches[0].distance:
            min_distance = matches[0].distance
            found_label = y_train_1[j]
    
    preds.append(found_label)

In [77]:
correct_matches = 0
for i in range(total_tests):
    if y_test_1[i] == preds[i]:
        correct_matches += 1

accuracy = (correct_matches / total_tests)*100
print(f"Accuracy : {accuracy: .2f} %")

Accuracy : 94%
